In [ ]:
import os
import sys
import random
import warnings

import numpy as np
#import pandas as pd

import matplotlib.pyplot as plt
from tensorflow import keras
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras import backend as K

import tensorflow as tf


# Set some parameters
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3





#TEST_PATH = 'input/modicyeni/test/images'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed

In [ ]:
TRAIN_PATH='E://mehmetyuksek//ircad-dataset//input/malign\\'
TEST_PATH='E:\\mehmetyuksek\\ircad-dataset\\input\\mtest\\'
# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]


In [ ]:
Deneme_Path_Train="E:\\saglikyenicalisma\\segmentation_models-master1\\segmentation_models-master\\examples\\deneme65\\malign\\"
Deneme_Path_Test="E:\\saglikyenicalisma\\segmentation_models-master1\\segmentation_models-master\\examples\\deneme65\\bening\\"
train_ids = next(os.walk(Deneme_Path_Train))[2]
test_ids = next(os.walk(Deneme_Path_Test))[2]

In [ ]:
test_ids

In [ ]:
## bilal hocanın dataseti için bunu kullanıyoruz

# Get and resize train images and masks
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = Deneme_Path_Train + id_
    img = imread(path)[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    #print(img.shape)
    X_train[n] = img
    

# Get and resize test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path1 = Deneme_Path_Test+id_
    img = imread(path1)[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img    

print('Done!')

In [ ]:
X_test.shape

In [ ]:
np.save("X_malign_5",X_train)
np.save("X_bening_5",X_test)

In [ ]:
## bilal hocanın dataseti için bunu kullanıyoruz

# Get and resize train images and masks
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH + id_
    img = imread(path + '/images/' + id_[2:] + '.png')[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    #print(img.shape)
    X_train[n] = img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                      preserve_range=True), axis=-1)
        #print(mask.shape)
        #print(mask_.shape)
        mask = np.maximum(mask, mask_)
        #print(mask)
    Y_train[n] = mask
    #print(Y_train[n])

# Get and resize test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_[2:] + '.png')[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
    Y_test[n] = mask

print('Done!')

In [ ]:

TEST_PATH='E:\\mehmetyuksek\\ircad-dataset\\input\\bmtest\\'
# Get train and test IDs

test_ids = next(os.walk(TEST_PATH))[1]

In [ ]:
# Get and resize test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_[2:] + '.png')[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
    Y_test[n] = mask

print('Done!')

In [ ]:
np.save("x_train_b.npy",X_train)

In [ ]:
np.save("y_train_b.npy",Y_train)

In [ ]:
np.save("x_test_b.npy",X_test)

In [ ]:
np.save("y_test_b.npy",Y_test)

In [ ]:
X_train=np.load("beningmaligndataset/x_train_bm.npy")

In [ ]:
Y_train=np.load("beningmaligndataset/y_train_bm.npy")

In [ ]:
X_test=np.load("beningmaligndataset/x_test_bm.npy")

In [ ]:
X_train.shape

In [ ]:
Y_test=np.load("beningmaligndataset/y_test_bm.npy")

In [ ]:
Y_test[0]

In [ ]:
 Y_train=Y_train.astype(int) 

In [ ]:
 Y_test=Y_test.astype(int) 

In [ ]:
import segmentation_models as sm

In [ ]:
BACKBONE = 'inceptionv3'
#efficientnetb0 seresnet101
BATCH_SIZE = 8
CLASSES = ['malignant']
LR = 0.0001
EPOCHS = 100
threshold1=0.5

#preprocess_input = sm.get_preprocessing(BACKBONE)


In [ ]:
X_train[0]

In [ ]:
# define network parameters
n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES) + 1)  # case for binary and multiclass segmentation
activation = 'sigmoid' if n_classes == 1 else 'softmax'

#create model
model = sm.FPN(BACKBONE, classes=n_classes, activation=activation)

In [ ]:
model.summary()

In [ ]:
# define optomizer
import keras
optim = keras.optimizers.Adam(LR)
    

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

# actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
# total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 

metrics = [sm.metrics.IOUScore(threshold=threshold1), sm.metrics.FScore(threshold=threshold1),sm.metrics.Precision(threshold=threshold1),sm.metrics.Recall(threshold=threshold1)]

# compile keras model with defined optimozer, loss and metrics
model.compile(optim, total_loss, metrics)

In [ ]:
# Fit model

"""callbacks = [
    #EarlyStopping(patience=50, verbose=1),
    #ReduceLROnPlateau(monitor='val_acc',factor=0.01, patience=15, min_lr=0.0001, verbose=1),
    ModelCheckpoint('hematadam05042021rmsprop.h5', verbose=1, save_best_only=True)]"""

callbacks = [
    keras.callbacks.ModelCheckpoint('./best_model_b_thres'+str(threshold1)+'_'+BACKBONE+'.h5', save_best_only=True, mode='max',verbose=1, monitor='val_iou_score')    
]
#callbacks = [EarlyStopping(patience=250, verbose=1),
#ModelCheckpoint('modelmalign08092020-a.h5', verbose=1, save_best_only=True)]
results = model.fit(X_train, Y_train, validation_split=0.20, batch_size=8, epochs=500,callbacks=callbacks,shuffle=True)

In [ ]:
# Plot training & validation iou_score values
plt.figure(figsize=(15, 5),dpi=300)
plt.subplot(121)
plt.plot(results.history['iou_score'])
plt.plot(results.history['val_iou_score'])
plt.title('Model iou_score')
plt.ylabel('iou_score')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation loss values
plt.subplot(122)
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
from classification_models.models_factory import ModelsFactory

In [ ]:
from classification_models

In [ ]:
ModelsFactory.

In [ ]:
BACKBONE="deneme"
def evaluateModel1(model, X_test, Y_test, batchSize):
    
    try:
        os.makedirs('results'+BACKBONE)
    except:
        pass 
    

    yp = model.predict(x=X_test, batch_size=batchSize, verbose=1)

    yp = np.round(yp,0)

    for i in range(len(Y_test)):

     
        image1_mask = np.array(yp[i].reshape(yp[i].shape[0],yp[i].shape[1]))
        image1_mask = np.ma.masked_where(image1_mask == 0, image1_mask)
        fig, ax = plt.subplots(1,4,figsize = (20,10))
        ax[0].imshow(X_test[i], cmap = 'gray')
        ax[0].set_title('Input')
        
        ax[1].imshow(Y_test[i].reshape(Y_test[i].shape[0],Y_test[i].shape[1]), cmap = 'gray')
        ax[1].set_title('Ground Truth')
        ax[2].imshow(image1_mask, cmap = 'gray')
        ax[2].set_title('Prediction')

        ax[3].imshow(X_test[i], cmap = 'gray', interpolation = 'none')
        ax[3].imshow(image1_mask, cmap = 'jet', interpolation = 'none', alpha = 0.7)
        ax[3].set_title('Last Segmentation')

        intersection = yp[i].ravel() * Y_test[i].ravel()
        union = yp[i].ravel() + Y_test[i].ravel() - intersection

        jacard = (np.sum(intersection)/np.sum(union))  
        fig.suptitle('Jacard Index:'+ str(np.sum(intersection)) +'/'+ str(np.sum(union)) +'='+str(jacard))
        
        
        

        fig.savefig('results'+BACKBONE+'/'+str(i)+'.png',format='png')
        


    jacard = 0
    dice = 0
    
    
    for i in range(len(Y_test)):
        yp_2 = yp[i].ravel()
        y2 = Y_test[i].ravel()
        
        intersection = yp_2 * y2
        union = yp_2 + y2 - intersection

        jacard += (np.sum(intersection)/np.sum(union))  

        dice += (2. * np.sum(intersection) ) / (np.sum(yp_2) + np.sum(y2))

    
    jacard /= len(Y_test)
    dice /= len(Y_test)
    


    print('Jacard Index : '+str(jacard))
    print('Dice Coefficient : '+str(dice))

In [ ]:
BACKBONE="deneme"
def evaluateModel2(model, X_test,  batchSize):
    
    try:
        os.makedirs('results'+BACKBONE)
    except:
        pass 
    

    yp = model.predict(x=X_test, batch_size=batchSize, verbose=1)

    yp = np.round(yp,0)

    for i in range(5):

     
        image1_mask = np.array(yp[i].reshape(yp[i].shape[0],yp[i].shape[1]))
        image1_mask = np.ma.masked_where(image1_mask == 0, image1_mask)
        fig, ax = plt.subplots(1,3,figsize = (20,10))
        ax[0].imshow(X_test[i], cmap = 'gray')
        ax[0].set_title('Input')
        
        
        ax[1].imshow(image1_mask, cmap = 'gray')
        ax[1].set_title('Prediction')

        ax[2].imshow(X_test[i], cmap = 'gray', interpolation = 'none')
        ax[2].imshow(image1_mask, cmap = 'jet', interpolation = 'none', alpha = 0.7)
        ax[2].set_title('Last Segmentation')

        
        

        fig.savefig('results'+BACKBONE+'/'+str(i)+'.png',format='png')
        


    

In [ ]:
res = load_model("E:/saglikyenicalisma/segmentation_models-master1/segmentation_models-master/examples/bilahocasonuclarsegmentation/bmsonuc/linknet/linknetbmseresnet/best_model_b_thres0.5_seresnet101.h5",compile=False)

In [ ]:
X_train=np.load("X_malign_5.npy")
X_test=np.load("X_bening_5.npy")

In [ ]:
res.summary()

In [ ]:
evaluateModel2(res, X_train, 5)

In [ ]:
evaluateModel1(model, X_train, Y_train, 5)

In [ ]:
evaluateModel1(model, X_train[50:100], Y_train[50:100], 5)

In [ ]:
model=load_model("best_model_Effnetb0.h5",custom_objects={'dice_loss_plus_1binary_focal_loss': total_loss,'iou_score':sm.metrics.IOUScore(threshold=0.5),'f1-score':sm.metrics.FScore(threshold=0.5)})

In [ ]:
yy=load_model("best_model_Effnetb0.h5")

In [ ]:
model.load_weights('best_model_resnet34.h5')

In [ ]:
import h5py
f = h5py.File('best_model_Effnetb0.h5', 'r')

In [ ]:
list(f.keys())

In [ ]:
dset = f['model_weights']

In [ ]:
dset

In [ ]:
dset.keys()

In [ ]:
cset=dset['training']

In [ ]:
cset

In [ ]:
cset.keys()